# SurpassAssist

---
Welcome to the Surpass Assist Demo Notebook. This interactive guide will provide a detailed overview of the functionalities of the Surpass Assist system, a state-of-the-art computer vision model designed to enhance road safety and traffic flow. By analyzing road conditions, detecting lanes, and tracking vehicles in front of large trucks, Surpass Assist collects valuable data to determine safe opportunities for vehicles to overtake. Through this demo, we aim to showcase how this technology can drastically reduce accidents and traffic jams caused by unsafe overtaking of large vehicles. Get ready to experience how Surpass Assist is revolutionizing road safety, merging advanced AI technology with practical road usage. Let's get started!

Please Note: That this notebook might contain slight bugs, that are easil fixable but time is short

**NOTE: PLEASE MAKE SURE GPU ACCELERATION IS TURNED ON -- Runtime -> Change Runtime Type**

**Make Sure it is set to GPU**

In [ ]:
# Installation & Imports
!apt-get -qq install -y libsm6 libxext6 libxrender-dev
!pip install -q torch torchvision
!pip install -q opencv-python-headless


%pip install ultralytics numpy tqdm requests supervision opencv-python
import ultralytics
import numpy as np
import cv2
import torch
import requests
import supervision as sv
from google.colab import drive
from pathlib import Path
from tqdm.notebook import tqdm
from google.colab import drive


ultralytics.checks()

Ultralytics YOLOv8.0.131 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [ ]:
# Defining file paths
HOME = Path("/content/")
video_path = Path(HOME / "ScenicDrive1.mp4")
output_video_path = Path(HOME / "demo.mp4")

model_path = Path("yolov8x") # leave this as is no need for an absolute path, it will download it automatically

#Download ScenicDrive1.mp4, instead of manually uploading it every time
Note: It might take a little file to appear in contents, it is there but will not show in the GUI

In [ ]:
video_url = "https://github.com/zain-codes/Object-Detection-Yolo-Project1/raw/master/Videos/ScenicDrive1.mp4"
response = requests.get(video_url, stream=True)
total_size_in_bytes= int(response.headers.get('content-length', 0))
block_size = 1024 #1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
with open(str(video_path), 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
  print("ERROR, something went wrong")

  0%|          | 0.00/38.9M [00:00<?, ?iB/s]

#Defining the wanted classes to be tracked

In [ ]:
# The class IDs for the desired objects to detect
# (0) Person
# (1-8) Vehicales
# (14-23) Animals
classes = {
            0: 'person',
            1: 'bicycle',
            2: 'car',
            3: 'motorcycle',
            4: 'airplane',
            5: 'bus',
            6: 'train',
            7: 'truck',
            15: 'cat',
            16: 'dog',
            17: 'horse',
            18: 'sheep',
            19: 'cow',
}
# Get the list of class IDs needed (the keys of the defined dict)
class_indexes = list(classes.keys())

In [ ]:
# Customize the bounding box
box_annotator = sv.BoxAnnotator(thickness=2,text_thickness=2,text_scale=1)

# Let's assume the focal length of your camera (in pixels) and the actual width of the object of interest (in meters) is known
FOCAL_LENGTH = 700  # Replace with your value, This value depends on your specific camera and would need to be calibrated
ACTUAL_WIDTH = 1.95  # Average width of a car (in meters)

# Initiate render progress bar



**WITH DISTANCE ESTEMATION**

In [ ]:
def calculate_distance(pixel_width):
    # Calculate distance from the camera to the object
    distance = (ACTUAL_WIDTH * FOCAL_LENGTH) / pixel_width
    return distance

In [ ]:
def get_bounding_box_info(det):
    xyxy, mask, confidence, class_id, tracker_id = det
    pixel_width = xyxy[2] - xyxy[0]
    box_area = pixel_width * (xyxy[3] - xyxy[1])
    return xyxy, pixel_width, box_area, class_id, tracker_id

In [ ]:
def calculate_and_store_area(past_areas, tracker_id, box_area, N):
    past_areas.setdefault(tracker_id, []).append(box_area)
    past_areas[tracker_id] = past_areas[tracker_id][-N:]  # keep last N areas
    return past_areas

In [ ]:
def determine_direction(past_areas, tracker_id, N, box_area):
    if len(past_areas[tracker_id]) == N:
        avg_past_area = sum(past_areas[tracker_id]) / N
        direction = 'moving away' if box_area > avg_past_area else 'Incoming'
    else:
        direction = 'Direction unknown'
    return direction

In [ ]:
def update_last_frame_boxes(last_frame_boxes, tracker_id, xyxy):
    last_frame_boxes[tracker_id] = xyxy
    return last_frame_boxes

In [ ]:
def create_label(classes, class_id, direction, distance,surpassable):
  if surpassable:
    surpassable = "Safe"
  else:
    surpassable = "Unsafe"
    return f"{direction}, Distance: {distance:.2f}m - ({surpassable})"

In [ ]:
def is_surpassable(distance,direction):
  if (distance < 70 and direction == 'moving away' ):
    return False

  return True

Note:
Due to the Limitatations of Google Colab
You cannot run live previews of demos
So we have dedcided to have the demo fully render inside of Colab
and then you can download and view the baisc demo on your own machine.
The Render process might take a little while, it is still beter than running a computer vision model locally with slow hardware:

In [ ]:
def process_video(video_path, model_path,output_video_path, class_indexes, classes, box_annotator):
    #Video Renderer defnition
    video_capture = cv2.VideoCapture(str(video_path))
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(str(output_video_path), fourcc, fps, (frame_width, frame_height))

    model = ultralytics.YOLO(str(model_path))
    pbar = tqdm(total=total_frames-2, desc='Rendering video')

    past_areas = {}  # Format: {tracker_id: [list of past areas]}
    last_frame_boxes = {}
    box_areas = {}
    # Create a dictionary to store the directions for each object
    directions = {}
    N = 10


    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        result = model.predict(frame, classes=class_indexes, verbose=False, agnostic_nms=True)[0]
        detections = sv.Detections.from_yolov8(result)
        detections = detections[np.isin(detections.class_id, class_indexes)]
        new_labels = []

        for det in detections:
            xyxy, pixel_width, box_area, class_id, tracker_id = get_bounding_box_info(det)
            distance = calculate_distance(pixel_width)
            past_areas = calculate_and_store_area(past_areas, tracker_id, box_area, N)
            direction = determine_direction(past_areas, tracker_id, N, box_area)
            last_frame_boxes = update_last_frame_boxes(last_frame_boxes, tracker_id, xyxy)
            surpassable = is_surpassable(distance,direction)
            new_labels.append(create_label(classes, class_id, direction, distance,surpassable))

        frame = box_annotator.annotate(scene=frame, detections=detections, labels=new_labels)
        output_video.write(frame)
        pbar.update(1)

    pbar.close()
    video_capture.release()
    output_video.release()
    cv2.destroyAllWindows()

In [ ]:
process_video(video_path, model_path,output_video_path, class_indexes, classes, box_annotator)

Rendering video:   0%|          | 0/2087 [00:00<?, ?it/s]